# ComfyUI Installation Verification and Monitoring

This notebook helps you monitor the ComfyUI installation process and verify that all components (ComfyUI-Manager and custom nodes) are properly installed.

## What this notebook does:
1. **Lists and filters files** in the ComfyUI directory
2. **Checks file permissions** and makes scripts executable
3. **Executes shell scripts** and monitors their output  
4. **Monitors background processes** like the installation script
5. **Parses script output** to extract installation status

Use this to verify that the fixed installation script is working correctly!

## 1. Listing and Filtering Files in a Directory

Let's start by examining the ComfyUI directory structure and looking for key files and directories.

In [ ]:
import os
import glob
import stat
import subprocess
import time
from pathlib import Path

# Define key directories
COMFYUI_DIR = "/workspace/ComfyUI"
CUSTOM_NODES_DIR = f"{COMFYUI_DIR}/custom_nodes"
TEMPLATE_DIR = "/workspace/comfy_template"

def list_directory_contents(directory, pattern="*", show_permissions=True):
    """List directory contents with file metadata"""
    print(f"\n📁 Contents of {directory}")
    print("=" * 80)
    
    if not os.path.exists(directory):
        print(f"❌ Directory {directory} does not exist!")
        return []
    
    files = glob.glob(os.path.join(directory, pattern))
    files.sort()
    
    results = []
    for file_path in files:
        try:
            file_stat = os.stat(file_path)
            size = file_stat.st_size
            permissions = stat.filemode(file_stat.st_mode)
            is_dir = os.path.isdir(file_path)
            name = os.path.basename(file_path)
            
            icon = "📁" if is_dir else "📄"
            print(f"{icon} {permissions} {size:>10} bytes  {name}")
            
            results.append({
                'name': name,
                'path': file_path,
                'size': size,
                'permissions': permissions,
                'is_directory': is_dir
            })
        except Exception as e:
            print(f"❌ Error reading {file_path}: {e}")
    
    return results

# Check ComfyUI main directory
print("🔍 Checking ComfyUI installation...")
comfyui_contents = list_directory_contents(COMFYUI_DIR)

In [ ]:
# Check custom_nodes directory specifically
print("\n🔍 Checking custom_nodes directory...")
custom_nodes_contents = list_directory_contents(CUSTOM_NODES_DIR)

# Look for ComfyUI-Manager specifically
manager_path = f"{CUSTOM_NODES_DIR}/ComfyUI-Manager"
print(f"\n🔍 Checking for ComfyUI-Manager at {manager_path}")
if os.path.exists(manager_path):
    print("✅ ComfyUI-Manager directory found!")
    manager_contents = list_directory_contents(manager_path)
    
    # Check for key files
    key_files = ["__init__.py", "cm-cli.py", "main.py"]
    for key_file in key_files:
        file_path = f"{manager_path}/{key_file}"
        if os.path.exists(file_path):
            print(f"✅ {key_file} found")
        else:
            print(f"❌ {key_file} missing!")
else:
    print("❌ ComfyUI-Manager directory not found!")

# Check for other important custom nodes
important_nodes = [
    "ComfyUI-VideoHelperSuite",
    "ComfyUI-KJNodes", 
    "ComfyUI-GGUF",
    "ComfyUI_essentials",
    "was-node-suite-comfyui"
]

print("\n🔍 Checking for important custom nodes...")
for node in important_nodes:
    node_path = f"{CUSTOM_NODES_DIR}/{node}"
    if os.path.exists(node_path):
        print(f"✅ {node} found")
    else:
        print(f"❌ {node} missing!")

## 2. Checking and Modifying File Permissions

Let's check the permissions of key scripts and make sure they're executable.

In [ ]:
def check_file_permissions(file_path):
    """Check if a file exists and its permissions"""
    if not os.path.exists(file_path):
        return f"❌ File {file_path} does not exist"
    
    file_stat = os.stat(file_path)
    permissions = stat.filemode(file_stat.st_mode)
    is_executable = bool(file_stat.st_mode & stat.S_IEXEC)
    
    status = "✅ Executable" if is_executable else "❌ Not executable"
    return f"{status} - {permissions} - {file_path}"

def make_executable(file_path):
    """Make a file executable"""
    if not os.path.exists(file_path):
        print(f"❌ Cannot make executable: {file_path} does not exist")
        return False
    
    try:
        current_mode = os.stat(file_path).st_mode
        os.chmod(file_path, current_mode | stat.S_IEXEC)
        print(f"✅ Made executable: {file_path}")
        return True
    except Exception as e:
        print(f"❌ Failed to make executable: {e}")
        return False

# Check key script files
key_scripts = [
    f"{TEMPLATE_DIR}/flux_install.sh",
    f"{TEMPLATE_DIR}/start_services.sh", 
    f"{COMFYUI_DIR}/main.py"
]

print("🔍 Checking script permissions...")
for script in key_scripts:
    print(check_file_permissions(script))

# Check ComfyUI-Manager CLI if it exists
cm_cli_path = f"{CUSTOM_NODES_DIR}/ComfyUI-Manager/cm-cli.py"
if os.path.exists(cm_cli_path):
    print(f"\n🔍 ComfyUI-Manager CLI permissions:")
    print(check_file_permissions(cm_cli_path))

## 3. Executing Shell Scripts Programmatically

Let's run some commands to check the installation status and execute scripts.

In [ ]:
def run_command(command, timeout=30):
    """Run a shell command and capture output"""
    try:
        print(f"🔄 Running: {command}")
        result = subprocess.run(
            command, 
            shell=True, 
            capture_output=True, 
            text=True, 
            timeout=timeout
        )
        
        print(f"📤 Exit code: {result.returncode}")
        if result.stdout:
            print(f"📄 Output:\n{result.stdout}")
        if result.stderr:
            print(f"⚠️ Errors:\n{result.stderr}")
        
        return result
    except subprocess.TimeoutExpired:
        print(f"⏰ Command timed out after {timeout} seconds")
        return None
    except Exception as e:
        print(f"❌ Error running command: {e}")
        return None

# Check if Python packages are installed
print("🔍 Checking critical Python packages...")
packages_to_check = ["torch", "safetensors", "transformers", "accelerate"]

for package in packages_to_check:
    result = run_command(f"python -c 'import {package}; print(f\"{package} version: {{getattr({package}, \"__version__\", \"unknown\")}}\")' 2>/dev/null")
    if result and result.returncode == 0:
        print(f"✅ {package} is installed")
    else:
        print(f"❌ {package} is NOT installed")

# Check git status of ComfyUI
print("\n🔍 Checking ComfyUI git status...")
if os.path.exists(COMFYUI_DIR):
    os.chdir(COMFYUI_DIR)
    run_command("git log -1 --oneline", timeout=10)
    run_command("git status --porcelain", timeout=10)

## 4. Monitoring Background Processes

Let's check what processes are running and monitor the installation script.

In [ ]:
def check_running_processes():
    """Check for running installation-related processes"""
    print("🔍 Checking running processes...")
    
    # Look for installation-related processes
    processes_to_check = ["flux_install.sh", "python", "git", "pip"]
    
    for process in processes_to_check:
        result = run_command(f"pgrep -af {process} | head -10", timeout=5)
        if result and result.stdout.strip():
            print(f"✅ Found {process} processes running")
        else:
            print(f"📄 No {process} processes found")

def monitor_installation_markers():
    """Check installation markers to see what's completed"""
    markers_dir = "/workspace/.install_markers"
    print(f"\n🔍 Checking installation markers in {markers_dir}...")
    
    if os.path.exists(markers_dir):
        markers = list_directory_contents(markers_dir, "*.done", show_permissions=False)
        if markers:
            print("\n✅ Completed installation steps:")
            for marker in markers:
                print(f"  • {marker['name'].replace('.done', '')}")
        else:
            print("📄 No completion markers found yet")
    else:
        print("❌ Markers directory doesn't exist")

def check_install_logs():
    """Look for installation log files"""
    log_files = [
        "/workspace/sageattention_install.log",
        "/workspace/comfyui_install.log",
        "/workspace/installation.log"
    ]
    
    print("\n🔍 Checking for log files...")
    for log_file in log_files:
        if os.path.exists(log_file):
            print(f"✅ Found log: {log_file}")
            # Show last few lines
            result = run_command(f"tail -5 {log_file}", timeout=5)
        else:
            print(f"📄 No log file: {log_file}")

# Run the checks
check_running_processes()
monitor_installation_markers()
check_install_logs()

## 5. Parsing and Displaying Script Output

Let's monitor the installation script output and test ComfyUI startup.

In [ ]:
def test_comfyui_import():
    """Test if ComfyUI can be imported successfully"""
    print("🔍 Testing ComfyUI import...")
    
    if not os.path.exists(COMFYUI_DIR):
        print("❌ ComfyUI directory not found!")
        return False
    
    # Change to ComfyUI directory and test import
    original_dir = os.getcwd()
    try:
        os.chdir(COMFYUI_DIR)
        result = run_command("python -c 'import comfy.utils; print(\"✅ ComfyUI imports successfully\")'", timeout=30)
        
        if result and result.returncode == 0:
            print("✅ ComfyUI core modules can be imported!")
            return True
        else:
            print("❌ ComfyUI import failed!")
            return False
    finally:
        os.chdir(original_dir)

def check_custom_nodes_import():
    """Test if custom nodes can be imported"""
    print("\n🔍 Testing custom node imports...")
    
    if not os.path.exists(CUSTOM_NODES_DIR):
        print("❌ Custom nodes directory not found!")
        return
    
    # Test ComfyUI-Manager import
    manager_path = f"{CUSTOM_NODES_DIR}/ComfyUI-Manager"
    if os.path.exists(f"{manager_path}/__init__.py"):
        print("✅ ComfyUI-Manager has __init__.py - should be importable")
    else:
        print("❌ ComfyUI-Manager missing __init__.py")
    
    # Check other custom nodes
    custom_nodes = [d for d in os.listdir(CUSTOM_NODES_DIR) 
                   if os.path.isdir(os.path.join(CUSTOM_NODES_DIR, d))]
    
    print(f"\n📄 Found {len(custom_nodes)} custom node directories:")
    for node in custom_nodes:
        init_file = f"{CUSTOM_NODES_DIR}/{node}/__init__.py"
        if os.path.exists(init_file):
            print(f"  ✅ {node} (has __init__.py)")
        else:
            print(f"  📄 {node} (no __init__.py)")

def final_verification():
    """Run final verification checks"""
    print("\n" + "="*80)
    print("🎯 FINAL VERIFICATION SUMMARY")
    print("="*80)
    
    # Check main components
    checks = [
        ("ComfyUI main.py", os.path.exists(f"{COMFYUI_DIR}/main.py")),
        ("ComfyUI-Manager", os.path.exists(f"{CUSTOM_NODES_DIR}/ComfyUI-Manager")),
        ("Custom nodes directory", os.path.exists(CUSTOM_NODES_DIR)),
        ("VideoHelperSuite", os.path.exists(f"{CUSTOM_NODES_DIR}/ComfyUI-VideoHelperSuite")),
        ("KJNodes", os.path.exists(f"{CUSTOM_NODES_DIR}/ComfyUI-KJNodes")),
        ("GGUF nodes", os.path.exists(f"{CUSTOM_NODES_DIR}/ComfyUI-GGUF")),
    ]
    
    all_good = True
    for check_name, check_result in checks:
        status = "✅" if check_result else "❌"
        print(f"{status} {check_name}")
        if not check_result:
            all_good = False
    
    print("\n" + "="*80)
    if all_good:
        print("🎉 ALL CHECKS PASSED! ComfyUI installation appears complete!")
        print("🚀 You should now be able to start ComfyUI and see the Manager and custom nodes.")
    else:
        print("⚠️  Some components are missing. Check the installation logs.")
    print("="*80)

# Run all verification tests
test_comfyui_import()
check_custom_nodes_import()
final_verification()

## 📋 Usage Instructions

**Run the cells above in order to:**

1. **Monitor installation progress** - Check what's being installed
2. **Verify file permissions** - Ensure scripts are executable
3. **Test package imports** - Verify Python dependencies
4. **Check background processes** - Monitor installation script
5. **Verify final installation** - Confirm all components are present

## 🔧 Troubleshooting

**If ComfyUI-Manager is still missing:**
- Re-run the verification notebook to check installation status
- Look at the installation markers in `/workspace/.install_markers/`
- Check for error messages in the installation output

**If custom nodes are missing:**
- The script now force-reinstalls all nodes on every run
- Check the custom_nodes directory: `/workspace/ComfyUI/custom_nodes/`
- Verify the installation script completed successfully

**If safetensors error persists:**
- The script now force-reinstalls critical packages at startup
- Check that the packages are actually installed with the import test above

**Next steps after installation completes:**
1. Navigate to ComfyUI in your browser (port 8188)
2. Check that the Manager button appears in the interface
3. Verify that custom node types are available in the node menu

The fixed installation script should now work reliably! 🎉